# _Datasets_ de entrenamiento y testeo

In [1]:
import math
import os

import joblib
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

from config import DATA_PATH, MODELS_PATH, VISUALIZATIONS_PATH

In [2]:
os.makedirs(MODELS_PATH, exist_ok=True)

In [51]:
data_path = os.path.join(DATA_PATH, "session_speech.csv")
data = pd.read_csv(data_path, converters={"speech_lemma_pos":eval})
data.columns

Index(['name', 'vote', 'senator', 'province', 'party', 'party_family',
       'speaker', 'speech', 'speech_preprocessed', 'speech_lemmas',
       'speech_pos', 'speech_lemma_pos', 'speech_preprocessed_count',
       'speech_preprocessed_count_unique', 'speech_lemmas_count',
       'speech_lemmas_count_unique', 'speech_pos_count',
       'speech_pos_count_unique', 'speech_lemma_pos_count_unique'],
      dtype='object')

In [52]:
data.head(2)

,name,vote,senator,province,party,party_family,speaker,speech,speech_preprocessed,speech_lemmas,speech_pos,speech_lemma_pos,speech_preprocessed_count,speech_preprocessed_count_unique,speech_lemmas_count,speech_lemmas_count_unique,speech_pos_count,speech_pos_count_unique,speech_lemma_pos_count_unique
0,Ana Claudia Almirón,positivo,ALMIRÓN ANA CLAUDIA,CORRIENTES,ALIANZA FRENTE PARA LA VICTORIA,Frente para la victoria,Almirón,"Hace dos años, tuvimos una sesión histórica cu...",hace dos años tuvimos una sesión histórica cua...,hacer año tener una sesión histórica cuando de...,VERB NOUN VERB DET NOUN ADJ SCONJ VERB ADP ADJ...,"[(hacer, VERB), (año, NOUN), (tener, VERB), (u...",1723,602,1684,459,1684,10,462
1,Roberto Gustavo Basualdo,negativo,BASUALDO ROBERTO GUSTAVO,SAN JUAN,ALIANZA CAMBIEMOS SAN JUAN,Juntos por el cambio,Basualdo,"Gracias, señor presidente. Hoy es un día en el...",gracias señor presidente hoy es un día en el q...,gracia señor presidente hoy ser un día en el q...,NOUN NOUN NOUN ADV VERB DET NOUN ADP DET SCONJ...,"[(gracia, NOUN), (señor, NOUN), (presidente, N...",411,187,405,146,405,10,147


In [53]:
data = (
    data[(data.speech.notna()) & (~data.vote.isin(["abstención", "ausente"]))]
    .reset_index(drop=True)
)
data.shape

(199, 19)

In [54]:
data.vote.value_counts(normalize=True)

vote
positivo    0.557789
negativo    0.442211
Name: proportion, dtype: float64

## Separación en _train_ y _test_

In [7]:
X_train_index, X_test_index = train_test_split(
    data.index,
    test_size=.2,
    random_state=6300,
    shuffle=True,
    stratify=data.vote
)

In [8]:
for index, name in zip([X_train_index, X_test_index], ["entrenamiento", "testeo"]):
    print(f"** Cantidad de datos en conjunto de {name}: {index.shape[0]}")
    print("** Distribución de la variable target:")
    print(f"{data.loc[index, 'vote'].value_counts(normalize=True).to_frame()}", end="\n\n")

** Cantidad de datos en conjunto de entrenamiento: 159
** Distribución de la variable target:
          proportion
vote                
positivo    0.559748
negativo    0.440252

** Cantidad de datos en conjunto de testeo: 40
** Distribución de la variable target:
          proportion
vote                
positivo        0.55
negativo        0.45



In [9]:
INDEX = os.path.join(MODELS_PATH, "index")
os.makedirs(INDEX, exist_ok=True)

for file in ["X_train_index", "X_test_index"]:
    dataset = eval(file)
    dataset.to_series().to_csv(os.path.join(INDEX, f"{file}.csv"), header=None, index=False)

## _Encoding_ de variables

In [10]:
VECTORIZERS = os.path.join(MODELS_PATH, "vectorizers")
os.makedirs(VECTORIZERS, exist_ok=True)

### Predictoras

#### Etiquetas POS

Uso la clase `TfidfVectorizer` de `scikit-learn` porque me permite hacer el cálculo rápidamente.

La configuración utilizada permite calcular la frecuencia relativa de cada etiqueta en el documento en cuestión.

Solo se utilizan las etiquedas de adejetivos, adverbios, nombres y verbos, aunque el cálculo se realiza considerando el total de las etiquetas.

In [11]:
pos_vectorizer = TfidfVectorizer(
    norm="l1", use_idf=False, smooth_idf=False, sublinear_tf=False
)
X = pos_vectorizer.fit_transform(data.loc[X_train_index, "speech_pos"])
pos = (
    pd.DataFrame(X.toarray(), columns=map(str.upper, pos_vectorizer.get_feature_names_out()))
    [["ADJ", "ADV", "NOUN", "VERB"]]
)
pos.head()

,ADJ,ADV,NOUN,VERB
0,0.108696,0.038043,0.217391,0.168478
1,0.065022,0.067265,0.217489,0.186099
2,0.000000,0.000000,0.666667,0.000000
3,0.094304,0.073763,0.185808,0.167134
4,0.100000,0.000000,0.400000,0.100000


In [12]:
pos_vectorizer_path = os.path.join(VECTORIZERS, "pos_vectorizer.pkl")
_ = joblib.dump(pos_vectorizer, pos_vectorizer_path, compress=True)

#### Lemmas

In [13]:
STATS = os.path.join(VISUALIZATIONS_PATH, "stats")

##### Proporciones sin _stopwords_ (Zipf)

In [19]:
proportions = pd.read_csv(os.path.join(STATS, "proporciones_sin_stopwords_zipf.csv"))
proportions.head()

,word,total,diff,pos,neg
0,abajo_adv,0.693147,-0.000040,0.000052,0.000092
1,abandonada_adj,0.000000,0.000052,0.000052,0.000000
2,abandonado_adj,0.000000,0.000052,0.000052,0.000000
3,abandonar_verb,0.000000,-0.000092,0.000000,0.000092
4,abandono_noun,0.000000,-0.000092,0.000000,0.000092


In [21]:
proportions.shape

(4801, 5)

In [22]:
proportions["group"] = proportions["diff"].apply(lambda x:"pos" if x>= 0 else "neg")
proportions.head()

,word,total,diff,pos,neg,group
0,abajo_adv,0.693147,-0.000040,0.000052,0.000092,neg
1,abandonada_adj,0.000000,0.000052,0.000052,0.000000,pos
2,abandonado_adj,0.000000,0.000052,0.000052,0.000000,pos
3,abandonar_verb,0.000000,-0.000092,0.000000,0.000092,neg
4,abandono_noun,0.000000,-0.000092,0.000000,0.000092,neg


In [30]:
proportions_words = list()

for key, df in proportions.groupby("group"):
    if key == "neg":
        df.sort_values(by=["diff", key], ascending=[True,False], inplace=True)
    else:
        df.sort_values(by=["diff", key], ascending=[False,False], inplace=True)
    words = df.word.to_list()[:250]
    proportions_words.extend(words)

len(proportions_words)

500

In [57]:
data["speech_lemma_pos"] = (
    data
    .speech_lemma_pos
    .apply(lambda x: " ".join(["_".join(i) for i in x]))
)

In [59]:
proportions_vectorizer = CountVectorizer(
    vocabulary=proportions_words
)
X = proportions_vectorizer.fit_transform(data.loc[X_train_index, "speech_lemma_pos"])
proportions_lemmas = (
    pd
    .DataFrame(
        X.toarray(),
        columns=proportions_vectorizer.get_feature_names_out()
    )
)
proportions_lemmas.head()

,constitución_propn,concepción_noun,nacer_verb,artículo_noun,internacional_adj,humano_adj,tratado_noun,persona_noun,problema_noun,acuerdo_noun,...,patriarcal_adj,perspectiva_noun,trato_noun,familia_noun,trabajo_noun,mucho_adv,realizar_verb,democracia_noun,formar_verb,luchar_verb
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,0,1,1,0,0,0,2,1,0,...,0,0,0,0,0,1,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,1,0,2,2,1,2,1,3,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [60]:
proportions_vectorizer_path = os.path.join(VECTORIZERS, "proporciones_vectorizer.pkl")
_ = joblib.dump(proportions_vectorizer, proportions_vectorizer_path, compress=True)

### Variable _target_

In [61]:
le = LabelEncoder()

data["target"] = le.fit_transform(data.vote)

In [62]:
for value in data.vote.unique():
    print(f"Categoría {value} ---> {le.transform([value])[0]}")

Categoría positivo ---> 1
Categoría negativo ---> 0


In [63]:
le_path = os.path.join(MODELS_PATH, "labelencoder.pkl")
_ = joblib.dump(le, le_path)